In [1]:
# ! git clone https://github.com/ClaudioPaterniti/MinesweeperLearning.git

In [2]:
# ! pip install plotly-express

In [3]:
import os
os.chdir('/content/MinesweeperLearning')

In [1]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from src.model import MineSweeperDataset, PatchMLPModel, OnHotEncodingTransform, Game
from src.player import ThresholdPlayer

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

cuda


In [6]:
train = Game(16, 30, 99, n = 10000)
train.random_open(0.3)
train.random_flags(0.3)
test = Game(n = 100)
test.random_open(0.3)
test.random_flags(0.3)

In [7]:
patch_radius = 3
name = f'patch_mlp_7x7_256_halving'
transform = OnHotEncodingTransform(patch_radius)
training_data = MineSweeperDataset(train, transform, losing_moves_weight=10)
test_data = MineSweeperDataset(test, transform)
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)
model = PatchMLPModel(patch_radius, layers=[2**i for i in range(8, 0, -1)], device=device)
player = ThresholdPlayer(model, 0.05, 0.95)
optimizer = torch.optim.Adam(model.model.parameters(), lr=0.001)

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model.train(train_dataloader, optimizer)
    model.test(test_dataloader)

Epoch 1
-------------------------------
Epoch 2
-------------------------------
Epoch 3
-------------------------------
Epoch 4
-------------------------------
Epoch 5
-------------------------------


In [ ]:
reinforcing_iterations = 10
epochs = 2
for i in range(reinforcing_iterations):
    games = Game(16, 30, 99, n = 1000)
    games.random_open(0.3)
    player.play(games)
    print(f'average score: {np.average(games.score())}')
    training_data.mix(games)
    train_dataloader = DataLoader(training_data, batch_size=64)
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        model.train(train_dataloader, optimizer)
        model.test(test_dataloader)

step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30
step 31
step 32
step 33
Epoch 1
-------------------------------
Epoch 2
-------------------------------
step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30
step 31
step 32
step 33
step 34
step 35
step 36
step 37
step 38
step 39
step 40
step 41
step 42
step 43
step 44
step 45
step 46
step 47
step 48
step 49
step 50
step 51
step 52
step 53
step 54
step 55
step 56
step 57
step 58
step 59
step 60
step 61
Epoch 1
-------------------------------
Epoch 2
-------------------------------
step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
ste

In [10]:
model.save(os.path.join('weights', name + '.pth'))

In [11]:
import plotly_express as px
import pandas as pd
df = pd.DataFrame({'train': model.train_loss_log, 'test': model.test_loss_log})
px.line(df)